In [1]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage


In [2]:
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

Package openjdk-8-jdk-headless is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-8-jdk-headless' has no installation candidate
--2023-01-01 13:59:53--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 99.84.160.46, 99.84.160.20, 99.84.160.25, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|99.84.160.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247880 (242K) [binary/octet-stream]
Saving to: ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.03s   

2023-01-01 13:59:53 (7.95 MB/s) - ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-

In [4]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

NAME                   PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-project2index  GCE       4                                       RUNNING  us-central1-a
-rw-r--r-- 1 root root 247882 Jan  1 13:55 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

spark

In [6]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'elad_318640828'
full_path = f"gs://{bucket_name}/"
paths=[]

In [7]:
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

# Count number of wiki pages
parquetFile.count()

6348910

In [9]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [10]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp import InvertedIndex

In [12]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124


def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS


# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    final_tokens = []
    for tok in tokens:
        if tok not in all_stopwords:
            final_tokens.append(tok)
    tf_dict = Counter(final_tokens)
    returning = []
    for elem in tf_dict.items():
        returning.append((elem[0], (id, elem[1])))
    return returning


##################
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    finale = []
    finale = sorted(unsorted_pl, reverse=False)
    return finale


##################
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    finale = []
    finale = postings.map(lambda x: (x[0], len(x[1])))

    return finale


##################
def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''

    file_location_dictionary = {}
    lama = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(
        lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))

    return lama

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))

postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>50)
postings_filtered_anchor = postings_anchor.filter(lambda x: len(x[1])>50)

w2df_text = calculate_df(postings_filtered_text)
w2df_title = calculate_df(postings_filtered_title)
w2df_anchor = calculate_df(postings_filtered_anchor)

w2df_dict_text = w2df_text.collectAsMap()
w2df_dict_title = w2df_title.collectAsMap()
w2df_dict_anchor = w2df_anchor.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered_text).collect()
_ = partition_postings_and_write(postings_filtered_title).collect()
_ = partition_postings_and_write(postings_filtered_anchor).collect()

index_const_time = time() - t_start

# test index construction time
print(index_const_time)

23/01/01 14:45:02 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 9.0 in stage 15.0 (TID 1242) (cluster-project2index-w-1.c.myfirstgcp-370210.internal executor 24): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 418, in func
    return f(iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2234, in combine
    merger.mergeValues(iterator)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/shuffle.py", line 240

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 19 in stage 15.0 failed 4 times, most recent failure: Lost task 19.3 in stage 15.0 (TID 1280) (cluster-project2index-w-1.c.myfirstgcp-370210.internal executor 24): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 418, in func
    return f(iterator)
  File "/u

In [15]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v)

In [16]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [17]:
# collect all posting lists locations into one super-set
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_anchor[k].extend(v)

In [27]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_text
# write the global stats out
inverted.write_index('.', 'index_body')
# upload to gs
index_src = "index_body.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_body/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_body.pkl [Content-Type=application/octet-stream]...
/ [1 files][  6.2 MiB/  6.2 MiB]                                                
Operation completed over 1 objects/6.2 MiB.                                      


In [28]:
!gsutil ls -lh $index_dst

  6.22 MiB  2023-01-01T16:04:03Z  gs://elad_318640828/postings_gcp_body/index_body.pkl
TOTAL: 1 objects, 6519494 bytes (6.22 MiB)


In [29]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_text
# write the global stats out
inverted.write_index('.', 'index_title')
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_title.pkl [Content-Type=application/octet-stream]...
/ [1 files][  6.2 MiB/  6.2 MiB]                                                
Operation completed over 1 objects/6.2 MiB.                                      


In [30]:
!gsutil ls -lh $index_dst

  6.22 MiB  2023-01-01T16:04:07Z  gs://elad_318640828/postings_gcp_title/index_title.pkl
TOTAL: 1 objects, 6519494 bytes (6.22 MiB)


In [31]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_text
# write the global stats out
inverted.write_index('.', 'index_anchor')
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_anchor/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_anchor.pkl [Content-Type=application/octet-stream]...
/ [1 files][  6.2 MiB/  6.2 MiB]                                                
Operation completed over 1 objects/6.2 MiB.                                      


In [32]:
!gsutil ls -lh $index_dst

  6.22 MiB  2023-01-01T16:04:13Z  gs://elad_318640828/postings_gcp_anchor/index_anchor.pkl
TOTAL: 1 objects, 6519494 bytes (6.22 MiB)


In [25]:
#Compute PageRank for the entire English Wikipedia. Use your implementation for generate_graph function from Colab below.
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  edges = pages.flatMap(lambda x: set([(x[0], link[0]) for link in x[1]]))
  list_vertices = edges.map(lambda x: ((x[1], 0)))
  groupi = edges.groupByKey()
  keys_vertices = groupi.map(lambda x: (x[0], 0))
  vertices = (keys_vertices + list_vertices).groupByKey().map(lambda x: [x[0]])

  return edges, vertices

In [26]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

Py4JJavaError: An error occurred while calling o290.parquet.
: java.io.IOException: Error accessing gs://wikidata_preprocessed/*
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2221)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2108)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1091)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1065)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.getFileStatus(GoogleHadoopFileSystemBase.java:955)
	at org.apache.hadoop.fs.FileSystem.isDirectory(FileSystem.java:1708)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.sc